# 02 – Data Cleaning: Handling Missing Values & Fixing Inconsistencies


This notebook covers the data cleaning steps on the training dataset.  

We handle selected missing values, resolve inconsistencies, and save a cleaned version of the dataset for use in feature engineering and modeling in later notebooks.


In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../data/raw/train.csv")

In [3]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [5]:
data.isnull().sum()[data.isnull().sum() > 0].sort_values(ascending=False)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtExposure      38
BsmtFinType2      38
BsmtFinType1      37
BsmtCond          37
BsmtQual          37
MasVnrArea         8
MasVnrType         8
Electrical         1
dtype: int64

## Insights

These features have some missing values. To decide how to handle them, we first need to check whether they are **categorical** or **numerical**.

We also examined whether some features are unnecessary or difficult to fill.

Most of the missing values are related to **Garage** and **Basement** features. So we began by analyzing those.

There are **81 missing values** in the garage-related columns. This means **81 houses have no garage**. We will fill these values with **"No Garage"** to avoid problems during modeling.

Some basement-related features like **BsmtQual** and **BsmtCond** have **37 missing values**, which likely means **37 houses have no basement**. These values will be filled with **"NoBsmt"**.

However, **BsmtFinType2** and **BsmtExposure** have **38 missing values**, indicating that **one house has a basement**, but specific information is missing. In this case, we will use **"Unknown"** instead of **"NoBsmt"**.

The **Masonry veneer** and **Electrical** features contain a small number of missing values. Since they are categorical variables, we will review them individually and handle missing or inconsistent data accordingly.

**LotFrontage** has **259 missing values**. Because it depends on location, we filled it using the **median LotFrontage within each neighborhood**.

Some features like **PoolQC**, **MiscFeature**, and **Alley** have many missing values. This does not indicate an issue — it simply means **most houses do not have these features**. For example, if there is no pool, **PoolQC** is left blank. We will later fill these missing values with **"None"** or similar labels.



While examining **masonry veneer** related variables, some inconsistent situations were encountered:

In [6]:
# Check for missing values in MasVnrType and how many are None or NaN
data['MasVnrType'].value_counts(dropna=False)

None       864
BrkFace    445
Stone      128
BrkCmn      15
NaN          8
Name: MasVnrType, dtype: int64

In [7]:
# Check for missing values in MasVnrArea and how many are None or NaN
data['MasVnrArea'].value_counts(dropna=False)

0.0      861
NaN        8
72.0       8
108.0      8
180.0      8
        ... 
562.0      1
89.0       1
921.0      1
762.0      1
119.0      1
Name: MasVnrArea, Length: 328, dtype: int64

There are **861** rows with a **MasVnrArea** value of **0.0**, and **8 missing values** in this column.  
In contrast, the **MasVnrType** column contains **864** entries labeled as **"None"**, along with **8 missing values**.


In [8]:
data[data['MasVnrArea'].isnull()][['Id', 'MasVnrType', 'MasVnrArea']]

,Id,MasVnrType,MasVnrArea
234,235,NaN,NaN
529,530,NaN,NaN
650,651,NaN,NaN
936,937,NaN,NaN
973,974,NaN,NaN
977,978,NaN,NaN
1243,1244,NaN,NaN
1278,1279,NaN,NaN


We filled the rows where both **MasVnrType** and **MasVnrArea** were missing by assigning them the values **"None"** and **0.0**, respectively.  
This approach assumes that these houses do not have any masonry veneer.

In [9]:
data.loc[data['MasVnrArea'].isnull(), 'MasVnrType'] = 'None'
data.loc[data['MasVnrArea'].isnull(), 'MasVnrArea'] = 0.0

As it seems, the missing values in both features correspond to each other.  
However, there might be an issue with the rows where **MasVnrArea** is **0.0** while **MasVnrType** is labeled as **"None"**.  
We need to examine whether these values are logically consistent.

In [10]:
data[(data['MasVnrType'] == 'None') & (data['MasVnrArea'] != 0.0)][['Id', 'MasVnrType', 'MasVnrArea']]

,Id,MasVnrType,MasVnrArea
624,625,None,288.0
773,774,None,1.0
1230,1231,None,1.0
1300,1301,None,344.0
1334,1335,None,312.0


We found several inconsistent rows where **MasVnrType** was labeled as **"None"**,  
but **MasVnrArea** had a non-zero value. This situation is not logically valid,  
since a non-zero area implies that there is some type of masonry veneer.

To correct this:

- For rows with **MasVnrType = "None"** and a **large area (> 50)**,  
  we assumed the veneer type was most likely **"BrkFace"**, which is the most common.

- For rows with **very small area values (≤ 1)** and **type = "None"**,  
  we assumed these were likely entry errors and replaced the area with **0.0**.


In [11]:
big_area = (data['MasVnrType'] == 'None') & (data['MasVnrArea'] > 50)
data.loc[big_area, 'MasVnrType'] = 'BrkFace' 

small_area = (data['MasVnrType'] == 'None') & (data['MasVnrArea'] <= 1)
data.loc[small_area, 'MasVnrArea'] = 0.0

In [12]:
data[(data['MasVnrType'] != 'None') & (data['MasVnrArea'] == 0.0)][['Id', 'MasVnrType', 'MasVnrArea']]

,Id,MasVnrType,MasVnrArea
688,689,BrkFace,0.0
1241,1242,Stone,0.0


We also identified two rows where **MasVnrType** had a defined veneer type (**"BrkFace"** and **"Stone"**), but the **MasVnrArea** was **0.0**. This is not logically consistent, as a valid veneer type implies the presence of some area.

To correct this inconsistency, we filled these rows using the **mean MasVnrArea** calculated from all other rows that share the same veneer type and have non-zero areas.

- The missing **BrkFace** area was filled with the mean of all non-zero BrkFace entries.
- The same was done for **Stone**.


In [13]:
mean_brkface = data[(data['MasVnrType'] == 'BrkFace') & (data['MasVnrArea'] > 0)]['MasVnrArea'].mean()
data.loc[(data['MasVnrType'] == 'BrkFace') & (data['MasVnrArea'] == 0), 'MasVnrArea'] = mean_brkface

mean_stone = data[(data['MasVnrType'] == 'Stone') & (data['MasVnrArea'] > 0)]['MasVnrArea'].mean()
data.loc[(data['MasVnrType'] == 'Stone') & (data['MasVnrArea'] == 0), 'MasVnrArea'] = mean_stone

In [14]:
data['MasVnrType'].value_counts(dropna=False)

None       869
BrkFace    448
Stone      128
BrkCmn      15
Name: MasVnrType, dtype: int64

In [15]:
data['LotFrontage'] = data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

In [16]:
mode_val = data['Electrical'].mode()[0]
data['Electrical'].fillna(mode_val, inplace=True)

In [17]:
# We'll fill 'Unknown' only if the house has a basement (BsmtQual is not null)
data.loc[(data['BsmtQual'].notnull()) & (data['BsmtExposure'].isnull()), 'BsmtExposure'] = 'Unknown'

# Do the same for BsmtFinType2
data.loc[(data['BsmtQual'].notnull()) & (data['BsmtFinType2'].isnull()), 'BsmtFinType2'] = 'Unknown'

In [18]:
data['BsmtExposure'].value_counts(dropna=False)

No         953
Av         221
Gd         134
Mn         114
NaN         37
Unknown      1
Name: BsmtExposure, dtype: int64

After completing the cleaning process for selected features, we can recheck the missing values to confirm the changes.


In [19]:
# Check remaining missing values
data[["MasVnrType", "MasVnrArea", "LotFrontage", "Electrical"]].isnull().sum()

MasVnrType     0
MasVnrArea     0
LotFrontage    0
Electrical     0
dtype: int64

All targeted features have been cleaned successfully. No missing values remain in:
- MasVnrType
- MasVnrArea
- Electrical
- LotFrontage


In [21]:
data.isnull().sum()[data.isnull().sum() > 0].sort_values(ascending=False)

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
BsmtQual          37
BsmtCond          37
BsmtExposure      37
BsmtFinType1      37
BsmtFinType2      37
dtype: int64

The remaining missing values belong to features like **Garage** and **Basement**, which will be handled later before modeling.


In [20]:
df_cleaned = data.copy()
df_cleaned.to_csv('../data/processed/train_cleaned.csv', index=False)